In [8]:
import numpy as np
import cv2
import os

In [9]:
#formula for the scaling factor is taken from https://web.stanford.edu/class/cs231a/course_notes/03-epipolar-geometry.pdf
#((2N)/(ΣNi=1||xi − x¯||2))1/2

def Trans_mat(points):
    #detemine the transformation
    centroid = np.average(points,axis=0)
    tx = centroid[0]
    ty = centroid[1]
    
    #determine the scalig factor
    ax = ( (2*len(points)) / ( np.sum((points[:,0]-tx)**2)) )**0.5
    ay = ( (2*len(points)) / ( np.sum((points[:,1]-ty)**2)) )**0.5
    
    T = np.array(([ax,0,-tx],[0,ay,-ty],[0,0,1]))
    return T

In [10]:
#algo of normalization is taken from the Youtube lecture 13 of Dr. Mubarak Shah by UCFCRCV
def normalize_data(T,pts):
    norm_points = []
    for i in range(len(pts)):
        p = np.dot(T,pts[i].T)
        norm_points.append(p.T)
    
    norm_points = np.reshape(norm_points, (len(pts),3))
    return norm_points

In [11]:
def find_fundamental_matrix(shape, pts1, pts2):
    """
    Computes Fundamental Matrix F that relates points in two images by the:

        [u' v' 1] F [u v 1]^T = 0
        or
        l = F [u v 1]^T  -- the epipolar line for point [u v] in image 2
        [u' v' 1] F = l'   -- the epipolar line for point [u' v'] in image 1

    Where (u,v) and (u',v') are the 2D image coordinates of the left and
    the right images respectively.

    Inputs:
    - shape: Tuple containing shape of img1
    - pts1: Numpy array of shape (N,2) giving image coordinates in img1
    - pts2: Numpy array of shape (N,2) giving image coordinates in img2

    Returns:
    - F: Numpy array of shape (3,3) giving the fundamental matrix F
    """
    
 
    ###########################################################################
    # TODO: Your code here                                                    #
    ###########################################################################
    A = np.ones((len(pts1),9))
    
    #transform points to homogeneous coordiates
    pts1_h = np.append(pts1, np.ones((len(pts1),1)), axis=1)
    pts2_h = np.append(pts2, np.ones((len(pts1),1)), axis=1)
    
    #normalize the data
    T1 = Trans_mat(pts1)
    T2 = Trans_mat(pts2)
    pts1_norm = normalize_data(T1, pts1_h)
    pts2_norm = normalize_data(T2, pts2_h)
    
    #form A matrix, as Af=0 where f will be the fundamnetal matrix
    for i in range(len(pts1)):
        u1 = pts1_norm[i][0]
        v1 = pts1_norm[i][1]
        u2 = pts2_norm[i][0]
        v2 = pts2_norm[i][1]
        
        A[i,:] = [u1*u2, u1*v2, u1, v1*u2, v1*v2, v1, u2, v2, 1]
    
    #compute f by taking svd of A
    U,E,V = np.linalg.svd(A)
    f = V[:,-1]
    f = f.reshape((3,3))
    
    #normalize f by taking L1 norm as per the way described in UCFCRV Dr.Muabarak shah lecture 13
    l1_norm = max(f.sum(axis=0))
    #print(l1_norm)
    F_norm = f/l1_norm
    
    #Rank enforcemet: enforce the constraint det(F)=0 using SVD
    U,S,V = np.linalg.svd(F_norm)
    S[-1] = 0
    S = np.diag(S)
    F_norm = U@S@V
    
    #renormalize F 
    F = T2.T @ F_norm @ T1
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################
    return F, F_norm

In [12]:
def compute_epipoles(F):
    """
    Given a Fundamental Matrix F, return the epipoles represented in
    homogeneous coordinates.

    Check: e2@F and F@e1 should be close to [0,0,0]

    Inputs:
    - F: the fundamental matrix

    Return:
    - e1: the epipole for image 1 in homogeneous coordinates
    - e2: the epipole for image 2 in homogeneous coordinates
    """
    ###########################################################################
    # TODO: Your code here                                                    #
    ###########################################################################
    #epipole 1 ---->    F*e1 = 0.....after solving it and bring it in the form ax=b where a=3x2 and x=2x1 and b=3x1
    #a=[f1 f2]   x=[x]   b=[-f3]
    #  [f4 f5]     [y]     [-f6]
    #  [f7 f8]             [-f9]
    
    A1 = F[:,0:2]
    b1 = -(F[:,-1])
    
    #solve it by x= (A.T*A)-1 A.T *b
    e1 = np.dot((np.linalg.inv(A1.T@A1)), (np.dot(A1.T,b1)))
    e1 = np.append(e1,1)
    #U1,S1,V1 = np.linalg.svd(F, full_matrices=True)
    #e1=V1[-1] 
    
    #epipole 2 ----->  F.T * e2 = 0
    A2 = F.T[:,0:2]
    b2 = -(F.T[:,-1])
    
    #solve it by x= (A.T*A)-1 A.T *b
    e2 = np.dot((np.linalg.inv(A2.T@A2)), (np.dot(A2.T,b2)))
    e2 = np.append(e2,1)
    #U2,S2,V2 = np.linalg.svd(F.T, full_matrices=True)
    #e2=V2[-1]
    ###########################################################################
    #                             END OF YOUR CODE                            #
    ###########################################################################

    return e1, e2

In [13]:
#draw epipolar lines on an image
def draw_epipolar_lines(F, pts1, img2, title):
    pts1_h = np.append(pts1, np.ones((len(pts1),1)), axis=1)
    for i in range(len(pts1_h)):
        l2 = np.dot(F, pts1_h[i].T)    #l2(a,b,c) being the line(ax+by+c=0) in image2 coreesponding to a point in image1
        
        # determine coordinates of the line in 2nd image
        x0 = 0
        y0 = int(-(l2[2]/l2[1]))
        x1 = int(img2.shape[1])
        y1 = int(-(l2[2]+ x1*l2[0])/l2[1])
        line_color = tuple(np.random.randint(30,250,3).tolist())
        cv2.line(img2, (x0,y0), (x1,y1), line_color, 1)
        
    cv2.imwrite(title, img2)

In [14]:
if __name__ == '__main__':

    # You can run it on one or all the examples
    names = os.listdir("C:/Users/dell/Desktop/Computer vision/Assignment-5/data/task23")
    output = "C:/Users/dell/Desktop/Computer vision/Assignment-5/results/"

    if not os.path.exists(output):
        os.mkdir(output)

    for name in names:
        print(name)

        # load the information
        img1 = cv2.imread(os.path.join("C:/Users/dell/Desktop/Computer vision/Assignment-5/data/task23", name, "im1.png"))
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img2 = cv2.imread(os.path.join("C:/Users/dell/Desktop/Computer vision/Assignment-5/data/task23", name, "im2.png"))
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        data = np.load(os.path.join("C:/Users/dell/Desktop/Computer vision/Assignment-5/data/task23", name, "data.npz"))
        pts1 = data['pts1'].astype(float)
        pts2 = data['pts2'].astype(float)
        K1 = data['K1']
        K2 = data['K2']
        shape = img1.shape

        # you can check against this
        FCheck, _ = cv2.findFundamentalMat(pts1, pts2, cv2.FM_8POINT)

        #######################################################################
        # TODO: Your code here                                                #
        Fund_mat,Fund_norm = find_fundamental_matrix(shape, pts1, pts2)
        epipoles = compute_epipoles(Fund_mat)

        #plot epipoles on the required images
        if name == 'reallyInwards' or name== 'ztrans':
            x1 = int(epipoles[0][0])
            y1 = int(epipoles[0][1])
            epi_color = tuple(np.random.randint(30,240,3).tolist())
            img_e1 = cv2.circle(img1, (x1,y1), radius=0, color=epi_color, thickness=4)
            cv2.imwrite(output+name+'1_epipoles.jpg',img_e1)
            
            x2 = int(epipoles[1][0])
            y2 = int(epipoles[1][1])
            img_e2 = cv2.circle(img2, (x2,y2), radius=0, color=epi_color, thickness=4)
            cv2.imwrite(output+name+'2_epipoles.jpg',img_e2)
          
        #plot epipolar lines on images
        if name == 'temple' or name== 'reallyInwards' or name== 'ztrans':
            draw_epipolar_lines(Fund_mat, pts1, img2, output+name+'1_lines.jpg')
            draw_epipolar_lines(Fund_mat.T, pts2, img1, output+name+'2_lines.jpg')
        
        #print all the fundamntal matrices 
        print('cv2 F matrix\n',FCheck,'\n\n Normalized F matrix\n', Fund_norm,'\n epipoles\n',epipoles,'\n' )
        #######################################################################

inwards
cv2 F matrix
 [[ 1.92547301e-10  1.63241271e-06 -7.83193368e-04]
 [ 1.63140089e-06 -2.65051052e-09 -5.52727704e-03]
 [-7.82169604e-04  3.44331053e-03  1.00000000e+00]] 

 Normalized F matrix
 [[ 1.48617010e-04  7.12026606e-04 -4.20988082e-03]
 [ 5.83021726e-01  2.77601298e+00 -1.06035810e+00]
 [ 4.14347045e-01  1.17245958e-01  1.66139810e+03]] 
 epipoles
 (array([-5.24288e+05,  2.62144e+05,  1.00000e+00]), array([5.8720256e+07, 6.1440000e+04, 1.0000000e+00])) 

reallyInwards
cv2 F matrix
 [[ 1.65953428e-10  1.63099879e-06 -7.81929531e-04]
 [ 1.63107496e-06 -8.61409553e-10 -1.80346920e-03]
 [-7.81989837e-04 -2.81993399e-04  1.00000000e+00]] 

 Normalized F matrix
 [[-1.09887707e-07  2.13924909e-07 -1.74047532e-06]
 [-7.40188290e-04  1.44123327e-03  1.04275101e-03]
 [ 6.56890610e-05 -1.07175601e-04  9.98958989e-01]] 
 epipoles
 (array([ 0.00000e+00, -2.62144e+05,  1.00000e+00]), array([1.00663296e+08, 9.34400000e+04, 1.00000000e+00])) 

temple
cv2 F matrix
 [[-4.56467394e-07  2.4